In [190]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm



In [154]:
df = pd.read_csv('accepted_paper_podium_AMIA_IS24.csv')

In [155]:
#uncomment for title and abstract
texts = ('TITLE: ' + df.submission_TITLE + ' \n\nABSTRACT: '+df.submission_ABSTRACT).tolist()
#texts = 'TITLE: ' + df.submission_TITLE

In [176]:
kwds = df.KEYWORDS_SELECTION.apply(ast.literal_eval).tolist()

# GPT

In [156]:
import requests

url = "https://api.openai.com/v1/embeddings"
access_token = "sk-xxx" # please replace with your openai token
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {access_token}'
}


In [157]:
inputs = [{
    "input": t,
    "model": "text-embedding-ada-002"} for t in texts]

In [198]:
# embedding for title+abstract
gpt_embeds_title_abstract = np.array([requests.post(url, headers=headers, json=inp).json()['data'][0]['embedding'] for inp in tqdm(inputs)])

100%|█████████████████████████████████████████| 144/144 [00:43<00:00,  3.28it/s]


In [192]:
# embedding for keywords
gpt_embeds_keywords = []
for kl in tqdm(kwds):
    embeds = [requests.post(url, headers=headers,
                            json={"input": k,"model": "text-embedding-ada-002"}).json()['data'][0]['embedding'] for k in kl]
    gpt_embeds_keywords.append(np.array(embeds).mean(axis=0))

gpt_embeds_keywords = np.array(gpt_embeds_keywords)      
    

100%|█████████████████████████████████████████| 144/144 [02:22<00:00,  1.01it/s]


In [202]:
gpt_embeds = np.concatenate([gpt_embeds_title_abstract, gpt_embeds_keywords],axis=1)

In [203]:
# !pip install k-means-constrained

from k_means_constrained import KMeansConstrained
clf = KMeansConstrained(n_clusters=24,size_min=6,size_max=6,random_state=0)
df['GPT_cluster'] = clf.fit_predict(np.array(gpt_embeds))

# Llama
## https://github.com/SeanLee97/AnglE/tree/cf51cdf011db47c3be6f75b7253c33e11cb3dc81
## https://arxiv.org/abs/2309.12871

In [160]:
import os 
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

peft_model_id = 'SeanLee97/angle-llama-7b-nli-v2'
config = PeftConfig.from_pretrained(peft_model_id)
os.environ['TRANSFORMERS_CACHE'] = '/data/HF/'
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, cache_dir="/data/HF/", do_sample=True)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, cache_dir="/data/HF/", do_sample=True).bfloat16().cuda()
model = PeftModel.from_pretrained(model, peft_model_id, cache_dir="/data/HF/").cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/ylo7832/.conda/envs/LLAMA/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ylo7832/.conda/envs/LLAMA/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [204]:
def getLlamaEmbed(txt):
    tok = tokenizer([txt], return_tensors='pt')
    for k, v in tok.items():
        tok[k] = v.cuda()
    return model(output_hidden_states=True, **tok).hidden_states[-1][:, -1].float().detach().cpu().numpy()[0]
    

In [213]:
# embedding for keywords
llama_embeds_title_abstract = [getLlamaEmbed(txt) for txt in tqdm(texts)]


    


100%|█████████████████████████████████████████| 144/144 [00:58<00:00,  2.45it/s]


In [214]:
# embedding for keywords
llama_embeds_keywords = []
for kl in tqdm(kwds):
    embeds = [getLlamaEmbed(k) for k in kl]
    llama_embeds_keywords.append(np.array(embeds).mean(axis=0))

llama_embeds_keywords = np.array(llama_embeds_keywords)      


100%|█████████████████████████████████████████| 144/144 [01:20<00:00,  1.79it/s]


In [215]:
llama_embeds = np.concatenate([llama_embeds_title_abstract, llama_embeds_keywords],axis=1)

In [216]:
# !pip install k-means-constrained
from k_means_constrained import KMeansConstrained
clf = KMeansConstrained(n_clusters=24,size_min=6,size_max=6,random_state=0)
df['Llama_cluster'] = clf.fit_predict(np.array(llama_embeds))

In [217]:
df.to_csv('clustered_accepted_paper_podium.csv')